# Bot Trader de Bitcoin
Este notebook implementa un bot de trading algorítmico para Bitcoin (BTC) con lógica mejorada, indicadores técnicos y gestión básica de riesgo.

## 1. Importar librerías necesarias
Importamos las librerías requeridas para la obtención de datos, análisis, visualización y cálculo de indicadores técnicos.

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go

## 1.1 Cripto a Analizar
Selecciona a traves del índice del array, la criptomoneda a analizar

In [2]:
cripto_seleccionada = ['BTC-USD','UNI7083-KRW']

cripto_seleccionada_por_indice = cripto_seleccionada[1]

## 2. Obtención de datos históricos de BTC
Descargamos los datos históricos de la Criptomoneda en USD usando yfinance, con intervalos de 5 minutos para los últimos 7 días.

In [3]:
def importar_base_bitcoin():
    global df_bitcoin
    datos = yf.download(cripto_seleccionada_por_indice, period='7d', interval='5m', auto_adjust=True)
    df_bitcoin = pd.DataFrame(datos)
    df_bitcoin.index.name = 'Datetime'

In [4]:
importar_base_bitcoin()
df_bitcoin.tail()

[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,UNI7083-KRW,UNI7083-KRW,UNI7083-KRW,UNI7083-KRW,UNI7083-KRW
Datetime,,,,,
2025-08-13 23:00:00+00:00,16767.960938,16773.759766,16765.929688,16773.759766,130678784
2025-08-13 23:05:00+00:00,16790.162109,16790.162109,16778.466797,16778.466797,788004864
2025-08-13 23:10:00+00:00,16777.302734,16797.705078,16777.302734,16790.539062,533725184
2025-08-13 23:15:00+00:00,16755.148438,16777.707031,16755.148438,16777.707031,31064064
2025-08-13 23:20:00+00:00,16741.542969,16744.558594,16741.542969,16744.558594,0


## 3. Web Scraping: Precio y tendencia actual de BTC
Obtenemos el precio y la tendencia actual de BTC desde CoinMarketCap usando BeautifulSoup.

In [5]:
def extraer_tendencias():
    global precio_actual, tendencia
    url = 'https://coinmarketcap.com/'
    respuesta = requests.get(url)
    if respuesta.status_code == 200:
        s = BeautifulSoup(respuesta.content, 'lxml')
        html_row = s.find_all('tr')[1]
        elementos = html_row.find_all('span')
        precio_actual = float(elementos[1].text.replace('$', '').replace(',', ''))
        if elementos[2].next.attrs['class'][0] == 'icon-Caret-up':
            tendencia = 'Alta'
        else:
            tendencia = 'Baja'
    else:
        precio_actual, tendencia = np.nan, 'Desconocida'

In [6]:
extraer_tendencias()
print(f'Precio actual: ${precio_actual} | Tendencia: {tendencia}')

Precio actual: $123109.08 | Tendencia: Alta


## 4. Limpieza y simplificación de datos
Eliminamos columnas innecesarias, registros con volumen cero y duplicados.

In [7]:
def limpiar_datos():
    global df_resumen
    df_resumen = df_bitcoin[['Close', 'Volume']].copy()
    df_resumen.columns = pd.Index(['Close', 'Volume'])
    df_resumen.index.name = None
    df_resumen = df_resumen[df_resumen['Volume'] > 0]
    df_resumen = df_resumen.drop_duplicates()

In [8]:
limpiar_datos()
df_resumen.describe()

,Close,Volume
count,1156.000000,1.156000e+03
mean,15208.424146,4.103410e+09
std,774.594097,7.753100e+09
min,13368.969727,6.553600e+04
25%,14744.475586,7.262863e+08
50%,15301.773926,1.828258e+09
75%,15648.250977,4.482220e+09
max,16870.021484,1.589011e+11


## 5. Indicadores técnicos: Medias móviles y RSI
Calculamos la media móvil simple (SMA), la media móvil exponencial (EMA) y el RSI para la toma de decisiones.

In [9]:
def calcular_indicadores():
    global df_resumen
    df_resumen['SMA_50'] = df_resumen['Close'].rolling(window=50).mean()
    df_resumen['EMA_20'] = df_resumen['Close'].ewm(span=20, adjust=False).mean()
    delta = df_resumen['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df_resumen['RSI'] = 100 - (100 / (1 + rs))

In [10]:
calcular_indicadores()
df_resumen[['Close', 'SMA_50', 'EMA_20', 'RSI']].tail()

,Close,SMA_50,EMA_20,RSI
2025-08-13 22:55:00+00:00,16770.146484,16611.591133,16700.374690,61.389817
2025-08-13 23:00:00+00:00,16767.960938,16612.136406,16706.811475,56.427203
2025-08-13 23:05:00+00:00,16790.162109,16615.600430,16714.749631,67.887330
2025-08-13 23:10:00+00:00,16777.302734,16619.077969,16720.707069,63.628806
2025-08-13 23:15:00+00:00,16755.148438,16623.403711,16723.987200,64.352679


In [11]:
df_resumen.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1156 entries, 2025-08-07 00:05:00+00:00 to 2025-08-13 23:15:00+00:00
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Close   1156 non-null   float64
 1   Volume  1156 non-null   int64  
 2   SMA_50  1107 non-null   float64
 3   EMA_20  1156 non-null   float64
 4   RSI     1143 non-null   float64
dtypes: float64(4), int64(1)
memory usage: 54.2 KB


## 6. Estrategia de trading
La decisión de trading se basa en la relación entre el precio actual, las medias móviles y el RSI. Se añade un umbral para evitar señales falsas y se recomienda gestión de riesgo.

In [12]:
def tomar_decision(umbral=0.002, rsi_compra=35, rsi_venta=65):
    global decision
    precio = df_resumen['Close'].iloc[-1]
    sma = df_resumen['SMA_50'].iloc[-1]
    ema = df_resumen['EMA_20'].iloc[-1]
    rsi = df_resumen['RSI'].iloc[-1]
    if (precio > sma * (1 + umbral)) and (precio > ema) and (rsi > rsi_venta):
        decision = 'Vender'
    elif (precio < sma * (1 - umbral)) and (precio < ema) and (rsi < rsi_compra):
        decision = 'Comprar'
    else:
        decision = 'Esperar'

In [13]:
tomar_decision()
print(f'Decisión de trading: {decision}')

Decisión de trading: Esperar


## 7. Visualización de precios, indicadores y decisión
Mostramos el precio, las medias móviles, el RSI y la decisión tomada en un gráfico interactivo.

In [14]:
def grafico_btc():
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df_resumen.index, y=df_resumen['Close'], name='Precio: ' + cripto_seleccionada_por_indice))
    fig.add_trace(go.Scatter(x=df_resumen.index, y=df_resumen['SMA_50'], name='SMA 50'))
    fig.add_trace(go.Scatter(x=df_resumen.index, y=df_resumen['EMA_20'], name='EMA 20'))
    color = 'green' if decision == 'Vender' else 'blue' if decision == 'Comprar' else 'orange'
    fig.add_annotation(
        x=df_resumen.index[-1], 
        y=df_resumen['Close'].iloc[-1], 
        text=decision, 
        showarrow=True, 
        font=dict(size=12, color=color), 
        arrowhead=2, arrowsize=1, arrowwidth=2, arrowcolor=color, ax=0, ay=-50, bordercolor=color, borderwidth=2, borderpad=4, bgcolor='white', opacity=0.8
    )
    fig.update_layout(title='Precio de '+cripto_seleccionada_por_indice+'; medias móviles y decisión', xaxis_title='Fecha', yaxis_title='Precio (USD)')
    fig.show()

In [15]:
grafico_btc()

In [16]:
print(f'Precio actual: ${precio_actual} | Tendencia: {tendencia} | Decisión: {decision}')

Precio actual: $123109.08 | Tendencia: Alta | Decisión: Esperar


## 8. Automatización del análisis
Puedes automatizar el análisis repitiendo el proceso cada cierto tiempo, usando un bucle y control de interrupciones.

In [17]:
import time
from IPython.display import clear_output
try:
    while True:
        clear_output()
        importar_base_bitcoin()
        extraer_tendencias()
        limpiar_datos()
        calcular_indicadores()
        tomar_decision()
        grafico_btc()
        print(f'Precio actual: ${precio_actual} | Tendencia: {tendencia} | Decisión: {decision}')
        time.sleep(300)
except KeyboardInterrupt:
    print('Ejecución detenida por el usuario.')

[*********************100%***********************]  1 of 1 completed


Precio actual: $123109.08 | Tendencia: Alta | Decisión: Esperar
Ejecución detenida por el usuario.
